In [ ]:
## Experience is a discrete scale from 1-5. 
# 1 is an Out of Domain (OOD) individual whose never labeled an image before
# 2 is an Out of Domain (OOD) individual who labeled more than 25 images | In Domain (ID) individual with no exerience
# 3 is an Out of Domain (OOD) individual who labeled more than 100 images | ID invidual with 25 images
# 4 is an Out of Domain (OOD) individual who labeled more than 500 images | ID individual with 100 images 
# 5 is an Out of Domain (OOD) individual who labeled more than 1000 images | ID invidual with 500 images


# Intialize User with self-ranking: In Domain v Out of Domain & approximate number of images labeled
# update user after each session based on their new # of images labeled.
weight_count = label_table['imID', 'WeightedCount'].groupby('ImID').sum()
    

def sorting_function(uncertainty, weighted_count, consensus_limit=10, r = 4):
    if weighted_count > consensus_limit: # this image doesn't need any more labels
        return -1
#     new_score = uncertainty * (1- r)^weighted_count # geometric decay, r = 0.29
    new_score = uncertainty * e^(-r*weighted_count) # exponential decay, lambda = 4

    return new_score

In [44]:
import pandas as pd
import numpy as np
df = pd.read_parquet("inventory_df_with_scores.parquet.gzip")

In [45]:
df.head()

,Index,image_path,pred_label,pred_class,index,entropy_score,least_confident_score,least_margin_score
0,0,/Users/alisonchase/Documents/IFCB/NAAMES/NAAME...,7,Other,0,0.201498,0.042467,0.077968
1,1,/Users/alisonchase/Documents/IFCB/NAAMES/NAAME...,7,Other,1,0.817881,0.281149,0.485377
2,2,/Users/alisonchase/Documents/IFCB/NAAMES/NAAME...,9,None,2,0.676992,0.391791,0.782708
3,3,/Users/alisonchase/Documents/IFCB/NAAMES/NAAME...,7,Other,3,0.089775,0.015530,0.028662
4,4,/Users/alisonchase/Documents/IFCB/NAAMES/NAAME...,7,Other,4,0.706994,0.354641,0.698348


In [46]:
df = df[['index', 'entropy_score']]

In [47]:
df.sort_values(by = 'entropy_score', ascending=False)

,index,entropy_score
968299,1015339,2.067445e+00
256206,256526,1.952166e+00
722824,723784,1.918797e+00
961125,1008165,1.903576e+00
670965,671925,1.896528e+00
...,...,...
287465,287785,6.000000e-08
441943,442263,6.000000e-08
349822,350142,6.000000e-08
978696,1025736,6.000000e-08


In [100]:
def get_view(df, label_df, lda= 4):
    #get counts
    most_recent_counts = label_df[['Im_ID', 'W_Count']].groupby('Im_ID').sum().reset_index()
    view_df = df.merge(most_recent_counts, left_on='index', right_on='Im_ID', how='left').drop(columns=['Im_ID'])
    view_df.W_Count = view_df.W_Count.fillna(0)
    view_df['sort_f'] = view_df.entropy_score* np.exp(-lda*view_df.W_Count)*(view_df.W_Count < 10)
    view_df = view_df.sort_values(by='sort_f', ascending=False)
    return view_df
    

In [51]:
import numpy as np

In [82]:
from tqdm.auto import trange, tqdm

In [149]:
label_df = pd.DataFrame(
    {
        "Im_ID": [],
        "U_UD" : [],
        "U_Label": [],
        "Session": [],
        "W_Count": []
    })

usrs = np.random.choice(np.arange(1,6,1), size=100, p= [0.1,0.1,0.3,0.2,0.3])
for i in trange(100):
    # images to label
    label_batch = get_view(df, label_df, lda=0.069).head(10)
    
    # assign random W_count:
    usr = usrs[i]
    W_count = [usr]*10
    Session = [i]*10
    
    tmp_df = pd.DataFrame({
        "Im_ID": label_batch['index'].values,
        "U_UD" : [usr]*10,
        "U_Label": np.random.choice(10, 10, replace=True),
        "Session": Session,
        "W_Count": W_count,
    })
    
    label_df = pd.concat(
        [label_df, tmp_df]
    )
    
    label_df['cum_W_count'] = label_df.groupby('Im_ID')['W_Count'].cumsum()
    label_df['cum_count'] = label_df.groupby('Im_ID').cumcount()

  0%|          | 0/100 [00:00<?, ?it/s]

In [102]:
label_df.shape

(2000, 5)

In [130]:
get_view(df, label_df, lda= 4)

,index,entropy_score,W_Count,sort_f
1044625,1091665,1.458342,0.0,1.458342e+00
719869,720829,1.457883,0.0,1.457883e+00
225846,226166,1.457829,0.0,1.457829e+00
31410,31410,1.457829,0.0,1.457829e+00
1050359,1097399,1.457810,0.0,1.457810e+00
...,...,...,...,...
454587,454907,1.466339,5.0,3.022351e-09
852530,899570,1.466254,5.0,3.022174e-09
311781,312101,1.466183,5.0,3.022028e-09
662675,663635,1.466175,5.0,3.022013e-09


In [150]:
label_df['Im_ID'].unique().shape

(950,)

In [148]:
np.exp(-0.069*10)

0.5015760690660555

In [147]:
np.log(0.5)/-10

0.06931471805599453

In [126]:
label_df['cum_count'].max()

0

In [151]:
label_df.drop_duplicates(subset=['Im_ID'], keep='last')

,Im_ID,U_UD,U_Label,Session,W_Count,cum_W_count,cum_count
0,938039.0,4.0,0.0,2.0,4.0,4.0,0
1,877015.0,4.0,6.0,2.0,4.0,4.0,0
2,621468.0,4.0,1.0,2.0,4.0,4.0,0
3,803830.0,4.0,6.0,2.0,4.0,4.0,0
4,434423.0,4.0,7.0,2.0,4.0,4.0,0
...,...,...,...,...,...,...,...
5,377134.0,3.0,6.0,99.0,3.0,3.0,0
6,769468.0,3.0,7.0,99.0,3.0,3.0,0
7,482818.0,3.0,2.0,99.0,3.0,3.0,0
8,1076716.0,3.0,4.0,99.0,3.0,3.0,0


In [153]:
label_df[label_df['cum_count'] == 1]

,Im_ID,U_UD,U_Label,Session,W_Count,cum_W_count,cum_count
4,1015339.0,3.0,7.0,11.0,3.0,6.0,1
9,256526.0,5.0,7.0,30.0,5.0,8.0,1
8,183933.0,5.0,6.0,32.0,5.0,6.0,1
6,367407.0,5.0,3.0,33.0,5.0,6.0,1
1,302750.0,3.0,5.0,34.0,3.0,4.0,1
0,214663.0,3.0,9.0,35.0,3.0,4.0,1
1,233957.0,3.0,6.0,35.0,3.0,4.0,1
4,436860.0,3.0,1.0,35.0,3.0,4.0,1
8,1083658.0,3.0,7.0,35.0,3.0,4.0,1
9,902479.0,3.0,4.0,35.0,3.0,4.0,1


In [152]:
label_df['cum_count'].unique()

array([0, 1], dtype=int64)

In [131]:
label_df_lda4 = label_df.copy()